## Notebook showing analysis of data for cell line E31

This motebook analyses the data data outputted from our cellProfiler pipeline for the cell line E31. The purpose of this notebook is to clearly show how this data was processed and analysed. Seperate python scripts exist for the application of our ML pipeline to drug discovery datasets. 

Functions called by this notebook can be found in "functions.py"

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import os
import umap
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import hdbscan
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.inspection import permutation_importance
from functions import *
import scipy.cluster.hierarchy as sch
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

##  Read in data and pre-process

**E_31_NucleiObject**: Data relating to the nucleus identified by the DAPI stain

**E_31_DilatedNuclei:** Data relating to a nulear shape slighly larger than the DAPI identified nucleus, done to capture the LaminB1 ring

**data_preprocessing_Nuclei:**
Takes a dataNucleiObject and process it, removing columns which relate to the location of the cell. We only want to keep the data for the DAPI stain and the nuclear morphology from these images, therefore we remove location data and columns relating to P21 and LaminB1.

**data_preprocessing_Dilated**: Takes a dataDilatedObject and process it, removing columns which relate to the location of the cell. We don't want to take the morphology data anymore as we've expanded the cell size to capture the P21 and LaminB1,so remove all data relating to nuclear DAPI stain.

In [ ]:
E_31_NucleiObject = pd.read_csv("/home/lucymartin/Documents/XDF/Cell_image_processing_code/Anna_data/final/E31_050423_P21_LaminB1NucleiObject.csv")
E_31_NucleiObject = data_processing_Nuclei(E_31_NucleiObject)
E_31_DilatedNuclei = pd.read_csv("/home/lucymartin/Documents/XDF/Cell_image_processing_code/Anna_data/final/E31_050423_P21_LaminB1DilatedNuclei_1.csv")
E_31_DilatedNuclei = data_processing_Dilated(E_31_DilatedNuclei)

In [ ]:
# create one data object containing all the data
data =  pd.concat([E_31_NucleiObject, E_31_DilatedNuclei], axis=1)

## Rescale intensity measures based on background levels

Want to scale all the control vaules to the same level and all the radiated to the same level? Could attempt to scale them all to the same thing but can't guarentee that the mask is masking all of the P21, so this might not be great

In [ ]:
E_31_image = pd.read_csv("/home/lucymartin/Documents/XDF/Cell_image_processing_code/Anna_data/final/E31_050423_P21_LaminB1Image.csv")

Plot before rescaling

In [ ]:
fig = px.histogram(data, x ='Intensity_MeanIntensity_CorrP21', color = 'ImageNumber', nbins = 2000)
fig.update_layout(
        font=dict(
            size=16,
        )
    )
fig.update_layout(
    title="P21 in each cell before normalisation",
    xaxis_title="Mean P21 intensity",
    yaxis_title="Count",
    barmode='overlay')
fig.update_traces(opacity=0.75)

fig.show()

In [ ]:
data = rescale_from_background(data, E_31_image)

Plot after rescaling

In [ ]:
fig = px.histogram(data, x ='Intensity_MeanIntensity_CorrP21', color = 'ImageNumber', nbins = 1000)
fig.update_layout(
        font=dict(
            size=16,
        )
    )
fig.update_layout(
    title="P21 in each cell after normalisation",
    xaxis_title="Mean P21 intensity",
    yaxis_title="Count",
    barmode='overlay')
fig.update_traces(opacity=0.75)

fig.show()

## Remove columns that are entirely NaN

In [ ]:
data = data.dropna(axis='columns')

## Remove cells that are an outlier in many catagories

For each cell want to calculate an outlier score - if a cell is outside of the 95th percentile in a load of the catagories then its probably not a cell 

In [ ]:
data = find_outliers(data, 70)

## Filter based on cell size

Removes any very small cells whih are likely to be artifacts and not real

In [ ]:
data = data[data['AreaShape_Area'] > 180]

## Filter based on Std dev

Removes any cells with a small DAPI std, which are likely to be out of focus

In [ ]:
data = data[data['Intensity_StdIntensity_CorrNuclei'] > 0.001]

## Create new features

In [ ]:
data = create_new_features(data)

## Look at histograms of P21 and LaminB1

In [ ]:
# line and shape = [x0, y0, x1, y1]
projection_line = [1., 0.025, 1.6, 0.005]
shaded = [0.5, 0.0115, 1.065, 0.16]
titles = ["E31", "Maximum value of the mean fractional LaminB1 intensity", "Mean intensity of P21"]

plot_hist_with_extras(data,'RadialDistribution_MeanFrac_CorrLaminB1_max', 'Intensity_MeanIntensity_CorrP21', projection_line, shaded, titles)

In [ ]:
# line and shape = [x0, y0, x1, y1]
line = []
shaded = []
titles = ["E31", "Maximum value of frac at distance * total intensity", "Mean intensity of P21"]

plot_hist_with_extras(data, 'RadialDistribution_MaxIntensFrac_CorrLaminB1_max', 'Intensity_MeanIntensity_CorrP21', line, shaded, titles)

In [ ]:
# line and shape = [x0, y0, x1, y1]
projection_line = [0, 0.03, 0.02, 0.005]
shaded = [0.0, 0.0115, 0.006, 0.16]
shaded_2 = [0.0063, 0, 0.06, 0.01]
titles = ["E31", "Mean intensity of LaminB1", "Mean intensity of P21"]

plot_hist_with_extras_2(data, 'Intensity_MeanIntensity_CorrLaminB1', 'Intensity_MeanIntensity_CorrP21', line, shaded, shaded_2, titles)

## Use this to add a column labelling those with low LaminB1 and high P21 as senescent

Thresholds determined by eye 

In [ ]:
Lam_cutoff = 0.006
P21_cutoff = 0.0115

data['Senescent'] = 0
data.loc[(data.Intensity_MeanIntensity_CorrLaminB1 < Lam_cutoff) & (data.Intensity_MeanIntensity_CorrP21 > P21_cutoff), 'Senescent'] = 1

In [ ]:
Lam_cutoff_1 = 0.0063
P21_cutoff_1 = 0.01

data['Not Senescent'] = 0
data.loc[(data.Intensity_MeanIntensity_CorrLaminB1 > Lam_cutoff_1) & (data.Intensity_MeanIntensity_CorrP21 < P21_cutoff_1), 'Not Senescent'] = 1

In [ ]:
print('Number of cells defined as senescent:')
print(sum(data['Senescent']))

In [ ]:
print('Number of cells defined as not senescent:')
print(sum(data['Not Senescent']))

## Project onto senescence axis

In [ ]:
data = project_onto_line(data, 'RadialDistribution_MeanFrac_CorrLaminB1_max', 'Intensity_MeanIntensity_CorrP21', projection_line)
#data, grad, c = project_onto_line_pca(data, 'Intensity_MeanIntensity_CorrLaminB1', 'Intensity_MeanIntensity_CorrP21')

## Create data_for_umap

In [ ]:
data_for_umap = data.dropna(axis='columns')
data_for_umap = data_for_umap.drop(['Metadata_CellLine', 'ImageNumber', 'ObjectNumber', 'Metadata_Radiated', 'Number_Object_Number', 'Senescent', 'Not Senescent', 'x_proj', 'y_proj'], axis = 1)

## Add a threshold in the data to remove features with low variance

In [ ]:
filter_threshold = 0.2
data_for_umap_filtered, filtered_columns = variance_threshold(data_for_umap, filter_threshold)

In [ ]:
filtered_columns[100:200]

## Format data for PCA

In [ ]:
# Drop column with nan as all of the entries
data_for_pca = data.dropna(axis='columns')
# replace an infinities with nan, then drop cells with nan
data_for_pca.replace([np.inf, -np.inf], np.nan, inplace=True)
data_for_pca = data_for_pca.dropna()

-----------------------

# Machine learning

## Split data into test and train

In [ ]:
y = data_for_pca["Senescent"]

Remove all data not DAPI related

In [ ]:
#remove everything not DAPI related
x = data_for_pca.copy()
x = x.drop(['Metadata_CellLine', 'ImageNumber', 'ObjectNumber', 'Metadata_Radiated', 'Number_Object_Number', 'Senescent', 'Not Senescent'], axis = 1)

In [ ]:
to_drop = []
for column in x.columns:
    split_cols = column.split('_')
    if len(split_cols) > 2:
        if split_cols[2] == 'CorrLaminB1' or split_cols[2] == 'CorrP21':
            to_drop.append(column)
x = x.drop(to_drop, axis = 1)

In [ ]:
x.columns

In [ ]:
print("Shape of x data")
print(x.shape)

split into test and train

In [ ]:
fraction_to_test = 0.5

In [ ]:
x_train_full, x_test_full, y_train, y_test = train_test_split(x, y, test_size=fraction_to_test)

drop the projection columns here so don't use them for ML but can still plot them 

In [ ]:
x_train = x_train_full.copy().drop(['x_proj', 'y_proj'], axis = 1)
x_test = x_test_full.copy().drop(['x_proj', 'y_proj'], axis = 1)

In [ ]:
x_test = StandardScaler().fit_transform(x_test)
x_train = StandardScaler().fit_transform(x_train)

In [ ]:
print("Shape of training data")
print(x_train.shape)

In [ ]:
print("Shape of testing data")
print(x_train_full.shape)

## SVM

In [ ]:
# selesct "balanced" option as have far fewer positively identified senescenet cells
clf_svm = svm.SVC(kernel='rbf', class_weight='balanced')

In [ ]:
clf_svm.fit(x_train, y_train)

In [ ]:
y_pred_svm = clf_svm.predict(x_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_svm))
# Model Precision: 
# What proportion of positive identifications were actually correct?
print("Precision:",metrics.precision_score(y_test, y_pred_svm))
# Model Recall:
# What proportion of actual positives were identified correctly?
print("Recall:",metrics.recall_score(y_test, y_pred_svm))

In [ ]:
pred_probs_svm = clf_svm.decision_function(x_test)
results_df = pd.DataFrame([pred_probs_svm, y_test]).T
results_df = results_df.sort_values(by=[0])

In [ ]:
fig = px.histogram(x = pred_probs_svm, title = "SVM senescence score distribution")
fig.update_layout(
        font=dict(
            size=16,
        )
    )
fig.update_layout(
        xaxis_title="Senescencen score",
        yaxis_title="Count")

fig.show()

In [ ]:
plot_ordered_classifier_score(results_df, "E31", "SVM")

In [ ]:
x_test_plot = x_test.copy()

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_test = tsne.fit_transform(x_test_plot)

In [ ]:
plot_projection("E31 TSNE on PCA coloured by SVM classifier score ", tsne_results_test, pred_probs_svm)

In [ ]:
plot_projection("E31 TSNE on PCA coloured by p21 senescence classification ", tsne_results_test, y_test)

## Test and train on sensesent and non-sen

This is subtly different, we now only test and train on the cells we classified as very senescent and very non-senescent with the DAPI stain. 

In [ ]:
# filter data for cells with only a score in either senescence or no senescence columns
x_2_catagories = data_for_pca.copy()
x_2_catagories = x_2_catagories[(x_2_catagories['Senescent'] == 1)|(x_2_catagories['Not Senescent'] == 1)]
y_2 = x_2_catagories["Senescent"]

In [ ]:
x_2_catagories = x_2_catagories.drop(['Metadata_CellLine', 'ImageNumber', 'ObjectNumber', 'Metadata_Radiated', 'Number_Object_Number', 'Senescent', 'Not Senescent', 'x_proj', 'y_proj'], axis = 1)
to_drop = []
for column in x_2_catagories.columns:
    split_cols = column.split('_')
    if len(split_cols) > 2:
        if split_cols[2] == 'CorrLaminB1' or split_cols[2] == 'CorrP21':
            to_drop.append(column)
x_2_catagories = x_2_catagories.drop(to_drop, axis = 1)

x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_2_catagories, y_2, test_size=fraction_to_test)

In [ ]:
x_train_2 = StandardScaler().fit_transform(x_train_2)
x_test_2 = StandardScaler().fit_transform(x_test_2)

**Train on the 2 subtypes, test on the two subtypes**

In [ ]:
clf_svm_2 = svm.SVC(kernel='rbf')
clf_svm_2.fit(x_train_2, y_train_2)
y_pred_svm_2 = clf_svm_2.predict(x_test_2)
pred_probs_svm_2 = clf_svm_2.decision_function(x_test_2)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test_2, y_pred_svm_2))
print("Precision:",metrics.precision_score(y_test_2, y_pred_svm_2))
print("Recall:",metrics.recall_score(y_test_2, y_pred_svm_2))

**train on the 2 subtypes, test on all**

In [ ]:
y_pred_svm_3 = clf_svm_2.predict(x_test)
pred_probs_svm_3 = clf_svm_2.decision_function(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_svm_3))
print("Precision:",metrics.precision_score(y_test, y_pred_svm_3))
print("Recall:",metrics.recall_score(y_test, y_pred_svm_3))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, pred_probs_svm)
roc_auc = auc(fpr, tpr)

fpr_2, tpr_2, thresholds_2 = roc_curve(y_test_2, pred_probs_svm_2)
roc_auc_2 = auc(fpr_2, tpr_2)

fpr_3, tpr_3, thresholds_3 = roc_curve(y_test, pred_probs_svm_3)
roc_auc_3 = auc(fpr_3, tpr_3)

display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
display_2 = RocCurveDisplay(fpr=fpr_2, tpr=tpr_2, roc_auc=roc_auc_2,estimator_name='example estimator')
display_3 = RocCurveDisplay(fpr=fpr_3, tpr=tpr_3, roc_auc=roc_auc_3,estimator_name='example estimator')
display.plot()
display_2.plot()
display_3.plot()
plt.show()

## Vary test and train split

In [ ]:
frac_test_array = np.arange(0.05, 1.0, 0.01)
frac_acc = []
frac_prec = []
frac_rec = []

for frac in frac_test_array:
    x_train_vary, x_test_vary, y_train_vary, y_test_vary = train_test_split(x_2_catagories, y_2, test_size=frac)
    # train on the 2 subtypes, test on the two subtypes
    x_train_vary = StandardScaler().fit_transform(x_train_vary)
    x_test_vary = StandardScaler().fit_transform(x_test_vary)
    clf_svm_vary = svm.SVC(kernel='rbf')
    clf_svm_vary.fit(x_train_vary, y_train_vary)
    y_pred_vary = clf_svm_vary.predict(x_test_vary)
    pred_probs_vary = clf_svm_vary.decision_function(x_test_vary)
    frac_acc.append(metrics.accuracy_score(y_test_vary, y_pred_vary))
    frac_prec.append(metrics.precision_score(y_test_vary, y_pred_vary))
    frac_rec.append(metrics.recall_score(y_test_vary, y_pred_vary))

frac_to_plot = pd.DataFrame(np.array([frac_test_array, frac_acc, frac_prec, frac_rec]).T, columns = ["frac to test", "accuracy", "precision", "recall"])

fig = go.Figure()
fig.add_trace(go.Scatter(x = frac_to_plot["frac to test"], y = frac_to_plot["accuracy"], mode = "markers", name = "Accuracy"))
fig.add_trace(go.Scatter(x = frac_to_plot["frac to test"], y = frac_to_plot["precision"], mode = "markers", name = "Precision"))
fig.add_trace(go.Scatter(x = frac_to_plot["frac to test"], y = frac_to_plot["recall"], mode = "markers", name = "Recall"))
fig.update_layout(
        font=dict(
            size=22,
        )
    )
fig.update_layout(
        title="E31 test train split",
        xaxis_title="Fraction of cells tested on",
        yaxis_title="Metric")

## Find important features

Use with caution - can take a long time to run! Finds the most import and features in the SVM model. 

In [ ]:
perm_importance = permutation_importance(clf_svm, x_test, y_test)

feature_names = x.columns
features = np.array(feature_names)

sorted_idx = perm_importance.importances_mean.argsort()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=features[sorted_idx][-20:],
    x=perm_importance.importances_mean[sorted_idx][-20:],
    name='SF Zoo',
    orientation='h',
    marker=dict(
        color='blue',
        line=dict(color='darkblue', width=3), opacity = 0.6
    )
))
fig.update_layout(
        font=dict(
            size=14,
        ),
    title = "E31 20 most important features in SVM",
    )

## Save x and y

In [ ]:
x = x.drop(['x_proj', 'y_proj'], axis = 1)

In [ ]:
# x.to_csv("E31_x.csv")

In [ ]:
# np.array(y).tofile('E31_y.csv',sep=',')

## Ada boost

Test the AdaBoost method, using many decision"stumps"

In [ ]:
clf_ada = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ada.fit(x_train, y_train)

In [ ]:
y_pred_ada = clf_ada.predict(x_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_ada))
# What proportion of positive identifications were actually correct?
print("Precision:",metrics.precision_score(y_test, y_pred_ada))
# What proportion of actual positives were identified correctly?
print("Recall:",metrics.recall_score(y_test, y_pred_ada))

In [ ]:
pred_probs_ada = clf_ada.decision_function(x_test)

In [ ]:
px.histogram(x = pred_probs_ada)

In [ ]:
results_df_ada = pd.DataFrame([pred_probs_ada, y_test]).T
results_df_ada = results_df_ada.sort_values(by=[0])

In [ ]:
plot_projection("E31 TSNE on PCA coloured by Ada boost classifier score ", tsne_results_test, pred_probs_ada)

**Train on the 2 subtypes, test on the two subtypes**

In [ ]:
clf_ada_2 = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ada_2.fit(x_train_2, y_train_2)
y_pred_ada_2 = clf_ada_2.predict(x_test_2)
print("Accuracy:",metrics.accuracy_score(y_test_2, y_pred_ada_2))
print("Precision:",metrics.precision_score(y_test_2, y_pred_ada_2))
print("Recall:",metrics.recall_score(y_test_2, y_pred_ada_2))

**Train on the 2 subtypes, test on all**

In [ ]:
y_pred_ada_3 = clf_ada_2.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_ada_3))
print("Precision:",metrics.precision_score(y_test, y_pred_ada_3))
print("Recall:",metrics.recall_score(y_test, y_pred_ada_3))

## Gradient boosting

Test GradientBoost method, using many decision trees

In [ ]:
clf_boost = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0,
                                 max_depth=5, random_state=0).fit(x_train, y_train)

In [ ]:
y_pred_boost = clf_boost.predict(x_test)
pred_probs_boost = clf_boost.decision_function(x_test)

In [ ]:
fig = px.histogram(x = pred_probs_boost, nbins=200,  title = "GradBoost senescence score distribution")
fig.update_layout(
        font=dict(
            size=16,
        )
    )
fig.update_layout(
        xaxis_title="Senescencen score",
        yaxis_title="Count")

fig.show()

In [ ]:
plot_projection("E31 TSNE on PCA coloured by gradient boost classifier score ", tsne_results_test, pred_probs_boost)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_boost))
# What proportion of positive identifications were actually correct?
print("Precision:",metrics.precision_score(y_test, y_pred_boost))
# What proportion of actual positives were identified correctly?
print("Recall:",metrics.recall_score(y_test, y_pred_boost))

**Train on the 2 subtypes, test on the two subtypes**

In [ ]:
clf_boost_2 = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0,
                                 max_depth=2, random_state=0).fit(x_train, y_train)
clf_boost_2.fit(x_train_2, y_train_2)
y_pred_boost_2 = clf_boost_2.predict(x_test_2)
print("Accuracy:",metrics.accuracy_score(y_test_2, y_pred_boost_2))
print("Precision:",metrics.precision_score(y_test_2, y_pred_boost_2))
print("Recall:",metrics.recall_score(y_test_2, y_pred_boost_2))

**Train on the 2 subtypes, test on all**

In [ ]:
y_pred_boost_3 = clf_boost_2.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_boost_3))
print("Precision:",metrics.precision_score(y_test, y_pred_boost_3))
print("Recall:",metrics.recall_score(y_test, y_pred_boost_3))

## How to best justify one of these classification choices?

We chose the SVM model as a continuous distribution os senscence scores best fit out desired application, and the metrics used indicated the SVM was performing well. 

In [ ]:
new_df_plot = pd.DataFrame(np.array([pred_probs_svm, pred_probs_ada, pred_probs_boost, x_test_full[:,-1], y_test]).T, columns = ['prediction', 'prediction ada', 'prediction boost', 'projection', 'senescent'])
target_mapping = {1: 'yes', 0: 'no'}
new_df_plot['senescent'] = new_df_plot['senescent'].map(lambda x: target_mapping[x])

In [ ]:
plot_continuous_classifier_comparison(new_df_plot, 'prediction', 'SVM')

In [ ]:
plot_continuous_classifier_comparison(new_df_plot, 'prediction ada', 'Ada')

In [ ]:
plot_continuous_classifier_comparison(new_df_plot, 'prediction boost', 'gradient boost')

In [ ]:
fig = px.scatter(x = data["x_proj"], y = data["y_proj"], color = data["Senescent"], opacity = 0.2)
fig.update_layout(
    font=dict(
        size=16,
    )
)
fig.update_layout(
    title="E31 projection onto P21 and LaminB1 axis",
    xaxis_title="Maximum value of the mean fractional LaminB1 intensity",
    yaxis_title="Mean intensity of P21")